<a href="https://colab.research.google.com/github/newmantic/BDT/blob/main/BDT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np

def generate_bdt_tree(volatility, forward_rates, time_steps):
    """
    Generate the Black-Derman-Toy tree for short rates.

    Args:
        volatility (float): Volatility of the short rate.
        forward_rates (list): List of forward rates for each time step.
        time_steps (int): Number of time steps in the tree.

    Returns:
        np.ndarray: A 2D array representing the BDT tree for short rates.
    """
    # Initialize the tree
    short_rate_tree = np.zeros((time_steps, time_steps))

    # Set the first node in the tree to the first forward rate
    short_rate_tree[0, 0] = forward_rates[0]

    # Populate the tree using the BDT model formula
    for i in range(1, time_steps):
        for j in range(i + 1):
            short_rate_tree[i, j] = forward_rates[i] * np.exp(-volatility * (i - 2 * j) * np.sqrt(1 / i))

    return short_rate_tree

In [3]:
def price_bond_bdt(face_value, coupon_rate, time_steps, short_rate_tree):
    """
    Price a bond using the Black-Derman-Toy model.

    Args:
        face_value (float): The face value of the bond.
        coupon_rate (float): The annual coupon rate of the bond.
        time_steps (int): Number of time steps in the BDT tree.
        short_rate_tree (np.ndarray): The BDT tree for short rates.

    Returns:
        float: The price of the bond.
    """
    # Initialize bond prices at maturity
    bond_prices = np.zeros((time_steps, time_steps))
    bond_prices[-1, :] = face_value + (coupon_rate * face_value)

    # Backward induction to calculate the bond price at each node
    for i in range(time_steps - 2, -1, -1):
        for j in range(i + 1):
            discount_factor = np.exp(-short_rate_tree[i, j])
            bond_prices[i, j] = (coupon_rate * face_value + 0.5 * (bond_prices[i + 1, j] + bond_prices[i + 1, j + 1])) * discount_factor

    return bond_prices[0, 0]

In [4]:
# Example inputs
volatility = 0.01
forward_rates = [0.03, 0.032, 0.034, 0.036, 0.038]
time_steps = len(forward_rates)

# Generate the BDT tree
bdt_tree = generate_bdt_tree(volatility, forward_rates, time_steps)
print("BDT Short Rate Tree:")
print(bdt_tree)

BDT Short Rate Tree:
[[0.03       0.         0.         0.         0.        ]
 [0.03168159 0.03232161 0.         0.         0.        ]
 [0.03352255 0.034      0.03448425 0.         0.        ]
 [0.03538183 0.03579275 0.03620845 0.03662897 0.        ]
 [0.03724755 0.03762189 0.038      0.03838191 0.03876765]]


In [5]:
# Example bond pricing inputs
face_value = 1000
coupon_rate = 0.05
time_steps = 5

# Use the previously generated BDT tree
bond_price = price_bond_bdt(face_value, coupon_rate, time_steps, bdt_tree)
print(f"Price of the Bond: {bond_price:.2f}")

Price of the Bond: 1104.91
